In [12]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [13]:
Hot_df = pd.read_excel('Datos/Ejercicio4.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df

,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,
H1,0,0,610,10,270,140,0,0,0,0,0
H2,0,0,0,0,270,140,350,110,420,0,0
H3,0,0,0,0,0,0,0,880,0,0,0
H4,0,0,0,0,0,0,0,0,1260,600,0


E igualmente de las frías.

In [14]:
Cold_df = pd.read_excel('Datos/Ejercicio4.xlsx', 'Cold',skiprows = 1,index_col=0)
Cold_df

,0,1,2,3,4,5,6,7,8,9,10
ID,,,,,,,,,,,
C1,0,0.000,0.000,25.000,675.000,350,875,275,1050,500,250
C2,0,1828.597,1828.597,29.977,809.379,0,0,0,0,0,0


In [15]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
model.h=Set(initialize=hotst)
model.c=Set(initialize=coldst)
model.i=Set(initialize=Hot_df.columns)

Con int_max calculamos el número total de intervalos

In [16]:
int_max=max(Hot_df.columns.values.tolist())
int_max

10

Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [17]:
model.pprint()

3 Set Declarations
    c : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    h : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'H1', 'H2', 'H3', 'H4'}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}

3 Declarations: h c i


In [18]:
Qs = model.Qs = Var(model.i,within = NonNegativeReals)
Qw = model.Qc = Var(model.i,within = NonNegativeReals)
R = model.R = Var(model.i,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [19]:
model.util = Objective(expr = model.Qs[1] + model.Qc[int_max])

Constraints

In [20]:
ni = list(model.i)[1:] 
model.int = ConstraintList()
for i in ni:
    model.int.add(
        R[i-1]+Qs[i]+sum(Hot_df[i]) == R[i]+sum(Cold_df[i])+Qw[i]
    )

model.R0 = Constraint(expr = R[0] == 0) 
model.R9 = Constraint(expr = R[int_max] == 0)

nii = list(model.i)[0:1]+list(model.i)[2:] 
model.steam = ConstraintList()
for i in nii:
    model.steam.add(
        Qs[i]==0
    )

niii = list(model.i)[0:int_max] 
model.cw = ConstraintList()
for i in niii:
    model.cw.add(
        Qw[i]==0
    )

Resolvemos el modelo

In [21]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

6 Set Declarations
    c : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'C1', 'C2'}
    cw_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    h : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'H1', 'H2', 'H3', 'H4'}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    int_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    steam_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9,

In [22]:
Qh = value(model.Qs[1])
Qc = value(model.Qc[int_max])
print('Cold utility = {0:2.1f}, Hot utility = {1:2.1f}'.format(Qc, Qh))

Cold utility = 1195.0, Hot utility = 4631.6
